In [17]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/diploma/data.csv


In [18]:
# импорт библиотек

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pandas_profiling
from pandas_profiling import ProfileReport
import os
import sys
import zipfile
import PIL
from PIL import ImageOps, ImageFilter
import re
import ast
from datetime import time, date, datetime
import random

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.style.use('seaborn-paper')

#from optuna import *

from sklearn import preprocessing
#from sklearn.feature_extraction import DictVectorizer
#from sklearn.feature_selection import SelectKBest, chi2, f_classif, f_regression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_predict, cross_val_score
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, MinMaxScaler, StandardScaler, RobustScaler
#from sklearn.impute import KNNImputer

#from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
#from sklearn.linear_model import Ridge
#from sklearn.svm import SVR
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.metrics import mean_squared_error, f1_score, accuracy_score, roc_curve, roc_auc_score
from sklearn import metrics
from sklearn.ensemble import StackingRegressor
from sklearn.ensemble import VotingRegressor
from tqdm.notebook import tqdm
from catboost import CatBoostRegressor, Pool, cv

#image augmentation
#import albumentations as A
#from ImageDataAugmentor.image_data_augmentor import *

# keras
import tensorflow as tf
import tensorflow.keras.backend as K
import tensorflow.keras.layers as L
from tensorflow.keras.models import Model, Sequential
#from tensorflow.keras.preprocessing.text import Tokenizer
#from tensorflow.keras.preprocessing import sequence
#from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
#from tensorflow.keras.applications.efficientnet import EfficientNetB3
#from tensorflow.keras.regularizers import l2
from tensorflow.keras.losses import MeanAbsolutePercentageError

#from transformers import AutoTokenizer, TFAutoModelForSequenceClassification, DataCollatorWithPadding

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')


pd.set_option('display.max_columns', 550) # больше колонок

os.environ['PYTHONHASHSEED']=str(42)

In [19]:
RANDOM_SEED = 42
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)
ITERATIONS = 200

In [20]:
DIR_DATA  = '../input/diploma/'

In [42]:
data_d = pd.read_csv(DIR_DATA+'data.csv')

In [22]:
def dollar_clean(string):
    i=0
    string_interm=re.findall('[0-9]+', string)
    
    if string_interm==[]:

        return np.NaN
    else:
        dollar_res=''
        while i<=(len(string_interm)-1):
            dollar_res=dollar_res + string_interm[i]
            i=i+1
        return (float(dollar_res))


In [23]:
data_d.target = data_d.target.apply(lambda x: dollar_clean(str(x)))

In [24]:
data_d['target'].isna().unique()

array([False,  True])

In [25]:
data_d.head(40)

,status,private pool,propertyType,street,baths,homeFacts,fireplace,city,schools,sqft,zipcode,beds,state,stories,mls-id,PrivatePool,MlsId,target
0,Active,NaN,Single Family Home,240 Heather Ln,3.5,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",Gas Logs,Southern Pines,"[{'rating': ['4', '4', '7', 'NR', '4', '7', 'N...",2900,28387,4,NC,NaN,NaN,NaN,611019,418000.0
1,for sale,NaN,single-family home,12911 E Heroy Ave,3 Baths,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",NaN,Spokane Valley,"[{'rating': ['4/10', 'None/10', '4/10'], 'data...","1,947 sqft",99216,3 Beds,WA,2.0,NaN,NaN,201916904,310000.0
2,for sale,NaN,single-family home,2005 Westridge Rd,2 Baths,"{'atAGlanceFacts': [{'factValue': '1961', 'fac...",yes,Los Angeles,"[{'rating': ['8/10', '4/10', '8/10'], 'data': ...","3,000 sqft",90049,3 Beds,CA,1.0,NaN,yes,FR19221027,2895000.0
3,for sale,NaN,single-family home,4311 Livingston Ave,8 Baths,"{'atAGlanceFacts': [{'factValue': '2006', 'fac...",yes,Dallas,"[{'rating': ['9/10', '9/10', '10/10', '9/10'],...","6,457 sqft",75205,5 Beds,TX,3.0,NaN,NaN,14191809,2395000.0
4,for sale,NaN,lot/land,1524 Kiscoe St,NaN,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",NaN,Palm Bay,"[{'rating': ['4/10', '5/10', '5/10'], 'data': ...",NaN,32908,NaN,FL,NaN,NaN,NaN,861745,5000.0
5,for sale,NaN,townhouse,1624 S Newkirk St,NaN,"{'atAGlanceFacts': [{'factValue': '1920', 'fac...",NaN,Philadelphia,"[{'rating': [], 'data': {'Distance': [], 'Grad...",897 sqft,19145,2 Beds,PA,2.0,NaN,NaN,PAPH847006,209000.0
6,Active,NaN,Florida,552 Casanova Ct,NaN,"{'atAGlanceFacts': [{'factValue': '2006', 'fac...",NaN,POINCIANA,"[{'rating': ['3', '3', '1', 'NR'], 'data': {'D...","1,507",34759,NaN,FL,One,NaN,NaN,S5026943,181500.0
7,Active,NaN,NaN,6094 Mingle Dr,NaN,"{'atAGlanceFacts': [{'factValue': '1976', 'fac...",NaN,Memphis,"[{'rating': ['4', '2', '2'], 'data': {'Distanc...",NaN,38115,NaN,TN,NaN,NaN,NaN,10063506,68000.0
8,Active,NaN,Single Family Home,11182 Owl Ave,2,"{'atAGlanceFacts': [{'factValue': '1970', 'fac...",NaN,Mason City,"[{'rating': ['2', '2', '4', '7', '4', 'NR'], '...",3588,50401,3,IA,NaN,NaN,NaN,190988,244900.0
9,NaN,NaN,Single Family,8612 Cedar Plains Ln,3,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",NaN,Houston,"[{'rating': ['4/10', '3/10', '2/10'], 'data': ...","1,930",77080,3,TX,2.0,NaN,NaN,73968331,311995.0


In [26]:
data_d['target'].fillna(value = data_d.target.median(), inplace = True)

In [27]:
data_d['target'].isna().unique()

array([False])

In [28]:
data_d['target'] = data_d[(data_d['target'] < 750000)]['target']

In [29]:
data_d['target'].isna().unique()

array([False,  True])

In [36]:
def facts_extraction(slovar):
    if slovar.find('atAGlanceFacts')>-1:
        slovar1 = eval(slovar)
       # print(type(slovar1))
        #print(slovar1)
        slovar2=slovar1.get('atAGlanceFacts')
       # print(type(slovar2))
       # print(slovar2)
        
        lst = ['']
        for i in slovar2:
            
            lst.append(i.get('factValue'))
        lst_res=lst[1:]
        return lst_res
            
    else: return np.NaN

In [41]:
r = "{'atAGlanceFacts': [{'factValue': '2008', 'factLabel': 'Year built'}, {'factValue': '', 'factLabel': 'Remodeled year'}, {'factValue': 'Forced Air', 'factLabel': 'Heating'}, {'factValue': 'Central', 'factLabel': 'Cooling'}, {'factValue': 'Attached Garage', 'factLabel': 'Parking'}, {'factValue': '', 'factLabel': 'lotsize'}, {'factValue': '$171/sqft', 'factLabel': 'Price/sqft'}]}"
print(type(r))

<class 'str'>


In [35]:
facts_extraction(r)

<class 'dict'>
{'atAGlanceFacts': [{'factValue': '2008', 'factLabel': 'Year built'}, {'factValue': '', 'factLabel': 'Remodeled year'}, {'factValue': 'Forced Air', 'factLabel': 'Heating'}, {'factValue': 'Central', 'factLabel': 'Cooling'}, {'factValue': 'Attached Garage', 'factLabel': 'Parking'}, {'factValue': '', 'factLabel': 'lotsize'}, {'factValue': '$171/sqft', 'factLabel': 'Price/sqft'}]}
<class 'list'>
[{'factValue': '2008', 'factLabel': 'Year built'}, {'factValue': '', 'factLabel': 'Remodeled year'}, {'factValue': 'Forced Air', 'factLabel': 'Heating'}, {'factValue': 'Central', 'factLabel': 'Cooling'}, {'factValue': 'Attached Garage', 'factLabel': 'Parking'}, {'factValue': '', 'factLabel': 'lotsize'}, {'factValue': '$171/sqft', 'factLabel': 'Price/sqft'}]


['2008', '', 'Forced Air', 'Central', 'Attached Garage', '', '$171/sqft']

In [49]:
t = data_d.loc[data_d['sqft']== 'Total interior livable area: 1,232 sqft']
t.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27 entries, 38 to 374019
Data columns (total 18 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   status        27 non-null     object
 1   private pool  2 non-null      object
 2   propertyType  27 non-null     object
 3   street        27 non-null     object
 4   baths         27 non-null     object
 5   homeFacts     27 non-null     object
 6   fireplace     7 non-null      object
 7   city          27 non-null     object
 8   schools       27 non-null     object
 9   sqft          27 non-null     object
 10  zipcode       27 non-null     object
 11  beds          27 non-null     object
 12  state         27 non-null     object
 13  stories       23 non-null     object
 14  mls-id        21 non-null     object
 15  PrivatePool   0 non-null      object
 16  MlsId         0 non-null      object
 17  target        27 non-null     object
dtypes: object(18)
memory usage: 4.0+ KB


In [51]:
t.head(27)

,status,private pool,propertyType,street,baths,homeFacts,fireplace,city,schools,sqft,zipcode,beds,state,stories,mls-id,PrivatePool,MlsId,target
38,For sale,NaN,Single Family,5531 Eureka Springs Rd,Bathrooms: 1,"{'atAGlanceFacts': [{'factValue': '1930', 'fac...",NaN,Tampa,"[{'rating': ['1/10', '2/10', '3/10'], 'data': ...","Total interior livable area: 1,232 sqft",33610,2 bd,FL,0.0,T3204536,NaN,NaN,"$117,900"
3800,For sale,NaN,Condo,1119 Freesia Ct,Bathrooms: 2,"{'atAGlanceFacts': [{'factValue': '2001', 'fac...",Yes,Ann Arbor,"[{'rating': ['8/10', '9/10'], 'data': {'Distan...","Total interior livable area: 1,232 sqft",48105,2 bd,MI,NaN,3267926,NaN,NaN,"$290,000"
20979,For sale,NaN,Single Family,59656 Glenmore Ave,Bathrooms: 2,"{'atAGlanceFacts': [{'factValue': '1987', 'fac...",Yes,Elkhart,"[{'rating': ['3/10', '6/10', '3/10'], 'data': ...","Total interior livable area: 1,232 sqft",46517,3 bd,IN,1.0,201941539,NaN,NaN,"$164,900"
21321,For sale,Yes,Condo,4545 Saint James Cir NW,Bathrooms: 2,"{'atAGlanceFacts': [{'factValue': '1982', 'fac...",Yes,Canton,"[{'rating': ['9/10', '6/10'], 'data': {'Distan...","Total interior livable area: 1,232 sqft",44708,2 bd,OH,1.0,4152443,NaN,NaN,"$114,900"
31938,For sale,NaN,Mobile / Manufactured,1905 Strasbourg Ln # 35,Bathrooms: 2,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",NaN,Antioch,"[{'rating': ['2/10', '1/10', '3/10'], 'data': ...","Total interior livable area: 1,232 sqft",94509,2 bd,CA,0.0,40879794,NaN,NaN,"$125,000"
40344,For sale,NaN,Single Family,57 Stage Coach Rd,Bathrooms: 2,"{'atAGlanceFacts': [{'factValue': '1976', 'fac...",NaN,Centerville,"[{'rating': ['NA', '4/10', '5/10'], 'data': {'...","Total interior livable area: 1,232 sqft",2632,3 bd,MA,1.0,21907567,NaN,NaN,"$359,900"
73878,For sale,NaN,Single Family,820 Brittany Dr,Bathrooms: 2,"{'atAGlanceFacts': [{'factValue': '1992', 'fac...",Yes,Denton,"[{'rating': ['6/10', '5/10', '5/10'], 'data': ...","Total interior livable area: 1,232 sqft",76209,3 bd,TX,1.0,14222112,NaN,NaN,"$205,000"
92412,For sale,NaN,Single Family,1281 Doe Ct SE,Bathrooms: 2,"{'atAGlanceFacts': [{'factValue': '1993', 'fac...",NaN,Palm Bay,"[{'rating': ['5/10', '4/10', '5/10'], 'data': ...","Total interior livable area: 1,232 sqft",32909,3 bd,FL,0.0,859197,NaN,NaN,"$184,000"
108164,For sale,NaN,Single Family,3328 Citadel Dr,Bathrooms: 2,"{'atAGlanceFacts': [{'factValue': '2011', 'fac...",NaN,Rockford,"[{'rating': ['4/10', '2/10', '1/10'], 'data': ...","Total interior livable area: 1,232 sqft",61109,3 bd,IL,1.0,201907080,NaN,NaN,"$149,000"
114549,For sale,NaN,Single Family,3019 Dearborn St,Bathrooms: 2,"{'atAGlanceFacts': [{'factValue': '1914', 'fac...",NaN,Youngstown,"[{'rating': ['2/10', '1/10'], 'data': {'Distan...","Total interior livable area: 1,232 sqft",44510,3 bd,OH,0.0,4141234,NaN,NaN,"$35,000"


In [52]:
t.homeFacts = t.homeFacts.apply(lambda x: facts_extraction(x))

In [54]:
t.head(10)

,status,private pool,propertyType,street,baths,homeFacts,fireplace,city,schools,sqft,zipcode,beds,state,stories,mls-id,PrivatePool,MlsId,target
38,For sale,NaN,Single Family,5531 Eureka Springs Rd,Bathrooms: 1,"[1930, 1930, Other, Central, , 2.14 acres lot,...",NaN,Tampa,"[{'rating': ['1/10', '2/10', '3/10'], 'data': ...","Total interior livable area: 1,232 sqft",33610,2 bd,FL,0.0,T3204536,NaN,NaN,"$117,900"
3800,For sale,NaN,Condo,1119 Freesia Ct,Bathrooms: 2,"[2001, , Forced air, Central, 1 space, -- sqft...",Yes,Ann Arbor,"[{'rating': ['8/10', '9/10'], 'data': {'Distan...","Total interior livable area: 1,232 sqft",48105,2 bd,MI,NaN,3267926,NaN,NaN,"$290,000"
20979,For sale,NaN,Single Family,59656 Glenmore Ave,Bathrooms: 2,"[1987, 1987, Forced air, Central, Attached Gar...",Yes,Elkhart,"[{'rating': ['3/10', '6/10', '3/10'], 'data': ...","Total interior livable area: 1,232 sqft",46517,3 bd,IN,1.0,201941539,NaN,NaN,"$164,900"
21321,For sale,Yes,Condo,4545 Saint James Cir NW,Bathrooms: 2,"[1982, 1982, Forced air, Central, 1 space, 9.9...",Yes,Canton,"[{'rating': ['9/10', '6/10'], 'data': {'Distan...","Total interior livable area: 1,232 sqft",44708,2 bd,OH,1.0,4152443,NaN,NaN,"$114,900"
31938,For sale,NaN,Mobile / Manufactured,1905 Strasbourg Ln # 35,Bathrooms: 2,"[, , Forced air, Central, Attached Garage, -- ...",NaN,Antioch,"[{'rating': ['2/10', '1/10', '3/10'], 'data': ...","Total interior livable area: 1,232 sqft",94509,2 bd,CA,0.0,40879794,NaN,NaN,"$125,000"
40344,For sale,NaN,Single Family,57 Stage Coach Rd,Bathrooms: 2,"[1976, 2019, Forced air, , , 0.38 acres lot, $...",NaN,Centerville,"[{'rating': ['NA', '4/10', '5/10'], 'data': {'...","Total interior livable area: 1,232 sqft",2632,3 bd,MA,1.0,21907567,NaN,NaN,"$359,900"
73878,For sale,NaN,Single Family,820 Brittany Dr,Bathrooms: 2,"[1992, 2005, Forced air, Central, 1 space, 0.2...",Yes,Denton,"[{'rating': ['6/10', '5/10', '5/10'], 'data': ...","Total interior livable area: 1,232 sqft",76209,3 bd,TX,1.0,14222112,NaN,NaN,"$205,000"
92412,For sale,NaN,Single Family,1281 Doe Ct SE,Bathrooms: 2,"[1993, 2008, Forced air, Central, Attached Gar...",NaN,Palm Bay,"[{'rating': ['5/10', '4/10', '5/10'], 'data': ...","Total interior livable area: 1,232 sqft",32909,3 bd,FL,0.0,859197,NaN,NaN,"$184,000"
108164,For sale,NaN,Single Family,3328 Citadel Dr,Bathrooms: 2,"[2011, , Forced air, Central, 4 spaces, 10,454...",NaN,Rockford,"[{'rating': ['4/10', '2/10', '1/10'], 'data': ...","Total interior livable area: 1,232 sqft",61109,3 bd,IL,1.0,201907080,NaN,NaN,"$149,000"
114549,For sale,NaN,Single Family,3019 Dearborn St,Bathrooms: 2,"[1914, 1989, Forced air, Central, 1 space, 0.2...",NaN,Youngstown,"[{'rating': ['2/10', '1/10'], 'data': {'Distan...","Total interior livable area: 1,232 sqft",44510,3 bd,OH,0.0,4141234,NaN,NaN,"$35,000"


In [53]:
t.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27 entries, 38 to 374019
Data columns (total 18 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   status        27 non-null     object
 1   private pool  2 non-null      object
 2   propertyType  27 non-null     object
 3   street        27 non-null     object
 4   baths         27 non-null     object
 5   homeFacts     27 non-null     object
 6   fireplace     7 non-null      object
 7   city          27 non-null     object
 8   schools       27 non-null     object
 9   sqft          27 non-null     object
 10  zipcode       27 non-null     object
 11  beds          27 non-null     object
 12  state         27 non-null     object
 13  stories       23 non-null     object
 14  mls-id        21 non-null     object
 15  PrivatePool   0 non-null      object
 16  MlsId         0 non-null      object
 17  target        27 non-null     object
dtypes: object(18)
memory usage: 4.0+ KB


In [55]:
data_d.homeFacts = data_d.homeFacts.apply(lambda x: facts_extraction(x))

In [57]:
data_d.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 377185 entries, 0 to 377184
Data columns (total 18 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   status        337267 non-null  object
 1   private pool  4181 non-null    object
 2   propertyType  342452 non-null  object
 3   street        377183 non-null  object
 4   baths         270847 non-null  object
 5   homeFacts     377185 non-null  object
 6   fireplace     103115 non-null  object
 7   city          377151 non-null  object
 8   schools       377185 non-null  object
 9   sqft          336608 non-null  object
 10  zipcode       377185 non-null  object
 11  beds          285903 non-null  object
 12  state         377185 non-null  object
 13  stories       226470 non-null  object
 14  mls-id        24942 non-null   object
 15  PrivatePool   40311 non-null   object
 16  MlsId         310305 non-null  object
 17  target        374704 non-null  object
dtypes: object(18)
memory usa

In [67]:
data_d.value_counts()

TypeError: unhashable type: 'list'

In [64]:
data_d_1 = pd.concat([data_d[['private pool','propertyType','street','baths','homeFacts',
                             'fireplace','city','schools','sqft','zipcode','beds','state','stories','mls-id','PrivatePool','MlsId','target']],
                      data_d['homeFacts'].str.split(',', expand=True)], axis=1)

In [65]:
data_d_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 377185 entries, 0 to 377184
Data columns (total 18 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   private pool  4181 non-null    object 
 1   propertyType  342452 non-null  object 
 2   street        377183 non-null  object 
 3   baths         270847 non-null  object 
 4   homeFacts     377185 non-null  object 
 5   fireplace     103115 non-null  object 
 6   city          377151 non-null  object 
 7   schools       377185 non-null  object 
 8   sqft          336608 non-null  object 
 9   zipcode       377185 non-null  object 
 10  beds          285903 non-null  object 
 11  state         377185 non-null  object 
 12  stories       226470 non-null  object 
 13  mls-id        24942 non-null   object 
 14  PrivatePool   40311 non-null   object 
 15  MlsId         310305 non-null  object 
 16  target        374704 non-null  object 
 17  0             0 non-null       float64
dtypes: f

In [66]:
data_d_1.head(3)

,private pool,propertyType,street,baths,homeFacts,fireplace,city,schools,sqft,zipcode,beds,state,stories,mls-id,PrivatePool,MlsId,target,0
0,NaN,Single Family Home,240 Heather Ln,3.5,"[2019, , Central A/C, Heat Pump, , , None, $144]",Gas Logs,Southern Pines,"[{'rating': ['4', '4', '7', 'NR', '4', '7', 'N...",2900,28387,4,NC,NaN,NaN,NaN,611019,"$418,000",NaN
1,NaN,single-family home,12911 E Heroy Ave,3 Baths,"[2019, , , , , 5828 sqft, $159/sqft]",NaN,Spokane Valley,"[{'rating': ['4/10', 'None/10', '4/10'], 'data...","1,947 sqft",99216,3 Beds,WA,2.0,NaN,NaN,201916904,"$310,000",NaN
2,NaN,single-family home,2005 Westridge Rd,2 Baths,"[1961, 1967, Forced Air, Central, Attached Gar...",yes,Los Angeles,"[{'rating': ['8/10', '4/10', '8/10'], 'data': ...","3,000 sqft",90049,3 Beds,CA,1.0,NaN,yes,FR19221027,"$2,895,000",NaN
